In [56]:
import numpy as np
import pandas as pd

pd.options.plotting.backend = 'plotly'
pd.options.display.float_format = '{:_.2f}'.format
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Задача и оценка
- Задача - предсказание цен на недвижимость в Мельбурне.
- Предсказание модели будет оцениваться по MAPE (mean absolute percentage error) - средняя абсолютная ошибка в процентах.

In [57]:
def bar(data, title='', showlegend=False):
    """Возвращает BarChart"""

    fig = px.bar(data, title=title)
    fig.update_xaxes(tickangle=-90)
    fig.update_layout(showlegend=showlegend)
    fig.show()

# Исследовательский анализ данных

### Общее знакомство с данными

In [58]:
train = pd.read_csv('../data/kaggle_leopard_challenge_train.csv')
train.columns = train.columns.str.lower()

In [59]:
train

,suburb,address,rooms,type,price,method,sellerg,date,distance,postcode,...,car,landsize,buildingarea,yearbuilt,councilarea,lattitude,longtitude,regionname,propertycount,id
0,Abbotsford,85 Turner St,2,h,1_480_000.00,S,Biggin,3/12/2016,2.50,3_067.00,...,1.00,202.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,4_019.00,34302
1,Abbotsford,25 Bloomburg St,2,h,1_035_000.00,S,Biggin,4/02/2016,2.50,3_067.00,...,0.00,156.00,79.00,1_900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4_019.00,33247
2,Abbotsford,5 Charles St,3,h,1_465_000.00,SP,Biggin,4/03/2017,2.50,3_067.00,...,0.00,134.00,150.00,1_900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4_019.00,31886
3,Abbotsford,40 Federation La,3,h,850_000.00,PI,Biggin,4/03/2017,2.50,3_067.00,...,1.00,94.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,4_019.00,18999
4,Abbotsford,55a Park St,4,h,1_600_000.00,VB,Nelson,4/06/2016,2.50,3_067.00,...,2.00,120.00,142.00,2_014.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4_019.00,16809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18368,Noble Park,5 Blaby St,3,h,627_500.00,PI,C21,30/09/2017,22.70,3_174.00,...,6.00,569.00,130.00,1_959.00,Greater Dandenong City Council,-37.97,145.18,South-Eastern Metropolitan,11_806.00,8052
18369,Reservoir,18 Elinda Pl,3,u,475_000.00,SP,RW,30/09/2017,12.00,3_073.00,...,1.00,NaN,105.00,1_990.00,Darebin City Council,-37.70,145.02,Northern Metropolitan,21_650.00,22511
18370,Roxburgh Park,14 Stainsby Cr,4,h,591_000.00,S,Raine,30/09/2017,20.60,3_064.00,...,2.00,NaN,225.00,1_995.00,Hume City Council,-37.64,144.93,Northern Metropolitan,5_833.00,31811
18371,Springvale South,30 Waddington Cr,3,h,780_500.00,S,Harcourts,30/09/2017,22.20,3_172.00,...,1.00,544.00,NaN,NaN,Greater Dandenong City Council,-37.98,145.15,South-Eastern Metropolitan,4_054.00,6855


In [60]:
bar(train.isna().mean(), 'train - % пропуска в данных')

In [61]:
corr = train.select_dtypes(include='number').corr()
corr['price'].sort_values(ascending=False)

price            1.00
rooms            0.50
bedroom2         0.48
bathroom         0.46
car              0.24
longtitude       0.20
postcode         0.10
buildingarea     0.09
landsize         0.04
id              -0.00
propertycount   -0.05
distance        -0.17
lattitude       -0.21
yearbuilt       -0.33
Name: price, dtype: float64

__Выводы__:
- Всего 8 переменных (из 22) имеют пропуски и кажется, что часть данных можно будет восстановить за счет имеющейся информации.
- Линейная корреляция показывает наличие `price` c ТОП3 пермеменными:
-   положительная связь: `rooms`, `bedroom2`, `bathroom`
-   отрицательная связь: `yearbuilt`, `lattitude`, `distance`
- Примечательно, что пости все переменные (с обнаруженной корреляцией) имеют пропуски. Возможно, что восстановив данные можно будет получить большее выраженный корреляционный эффект. Правда, не стоит забывать, что взаимосвязь между метриками может быть и не линейной. В таком случае корреляция нам ничего не покажет.

__План действий__:

Так как уже имеется BaseLine (CatBoost с датасетом "как есть"), то все дальнейшие действия с данными будут направлены на улучшение данного BaseLine:
- Обработать все метрики в имеющемся наборе данных.
- Сгенерировать новые метрики на основе имеющихся.
- Найти варианты/сочетания метрик и параметров модели дающих максимальный результат :)

## suburb


In [62]:
bar(train['suburb'].value_counts())

In [63]:
train['suburb'].value_counts(normalize=True)

Reservoir        0.03
Bentleigh East   0.02
Richmond         0.02
Preston          0.02
Brunswick        0.02
                 ... 
Wonga Park       0.00
Upwey            0.00
Bacchus Marsh    0.00
Rockbank         0.00
Monbulk          0.00
Name: suburb, Length: 329, dtype: float64

## address

Кажется, что данный признак можно сделать более точным если разделить на составляющие название улицы и адрес дома. Возможно, что через адрес получится восстановить значительную часть географисеских координат.

In [64]:
def get_address(data):
    """Создает две новых метрики (улица/номер дама) из address"""

    t = data['address'].str.split(' ', expand=True)

    def get_street(row):
        cell_one = row[1]
        cell_two = row[2]
        cell_three = row[3]

        # Обработка строк где cell_three == <NA>
        if pd.isna(cell_three):
            return cell_one + ' ' + cell_two
        else:
            # Обработка строк с дубликатами
            if cell_one == cell_three:
                return cell_two
            else:
                return cell_one + ' ' + cell_two + ' ' + cell_three

    t['street'] = t.apply(get_street, axis=1)

    t.rename(columns={0: 'house'}, inplace=True)

    data = data.merge(t[['house', 'street']], how='left', left_index=True, right_index=True)

    return data

In [65]:
# Добавим новые признаки в данные
train = get_address(train)

## rooms

In [66]:
train['rooms'].hist(title='Распределение rooms в данных')

### type

In [67]:
train['type'].hist(title='Распределение type в данных')

###  price

In [68]:
train['price'].hist(title='Распределение price в данных')

In [69]:
train['price'].describe()

count      18_373.00
mean    1_053_234.88
std       621_797.25
min       227_000.00
25%       635_000.00
50%       880_000.00
75%     1_301_000.00
max     5_580_000.00
Name: price, dtype: float64

## method

In [70]:
train['method'].hist()

In [71]:
t = train.pivot_table(index='method', aggfunc={'price': ['min', 'max', 'mean', 'sum']})
t.columns = ['_'.join(col) for col in t.columns]
t.sort_values('price_mean', ascending=False)

,price_max,price_mean,price_min,price_sum
method,,,,
VB,5_500_000.00,1_152_115.52,240_000.00,1_944_771_000.00
PI,5_300_000.00,1_099_629.37,230_000.00,2_396_092_399.00
S,5_525_000.00,1_065_913.84,227_000.00,12_817_613_918.00
SA,3_225_000.00,991_058.59,240_000.00,126_855_500.00
SP,5_580_000.00,877_922.51,240_000.00,2_065_751_674.00


## sellerg

In [72]:
train['sellerg'].value_counts(normalize=True)

Nelson          0.11
Jellis          0.10
hockingstuart   0.09
Barry           0.08
Ray             0.06
                ... 
Zahn            0.00
Homes           0.00
Allan           0.00
Steveway        0.00
Point           0.00
Name: sellerg, Length: 305, dtype: float64

In [73]:
# Интересно, а лидерство в количестве продаж равняется лидерству в объеме денег?
t = train.pivot_table(index='sellerg', aggfunc={'id': 'count', 'price': 'sum'})
t.sort_values('id', ascending=False, inplace=True)
t = t.head(30)

In [74]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=t.index, y=t['id'], name="количество продаж"), secondary_y=False)
fig.add_trace(go.Scatter(x=t.index, y=t['price'], name="сумма продаж"), secondary_y=True)
fig.update_layout(title_text="ТОП30. Количество продаж и сумма продаж.")
fig.update_xaxes(title_text="xaxis title", tickangle=-90)
fig.update_yaxes(title_text="количество продаж", secondary_y=False)
fig.update_yaxes(title_text="сумма продаж", secondary_y=True)
fig.show()

Похоже, что кто-то умеет продавать меньше, но дороже :)
Здесь появляется гипотеза, что цена уровень цены может зависеть не только от самого продаваемого объекта, но и от продавца. В настоящий момент времени нет желания углубляться в проверку этой гипотезы, но возможно, что вернемся к ней позже.

## date

In [75]:
# Преобразуем дату и сгенерируем дополнительные признаки
train['date'] = pd.to_datetime(train['date'], errors='coerce', format='%d/%m/%Y')
train['month'] = train['date'].apply(lambda x: x.replace(day=1))
train['day_name'] = train['date'].dt.day_name()

train['date'].isna().mean()

0.0

In [76]:
# В поисках возможных сезонных трендов посмотрим динамику продаж по дням
train.groupby('date')['id'].count().plot(kind='bar', title='Динамика продаж по дням')

In [77]:
# И по месяцам тоже
train.groupby('month')['id'].count().plot(kind='bar', title='Динамика продаж по месяцам')

In [78]:
def check_date(target):
    """target - pd.DataFrame[column]"""

    unique_day = target.nunique()
    min_day = target.dt.date.min()
    max_day = target.dt.date.max()
    period = max_day - min_day
    print(f'date period from {min_day} to {max_day}, total_days {period.days}\n'
          f'unique days in data: {unique_day} ({unique_day / period.days :.2%})')

In [79]:
check_date(train['date'])

date period from 2016-01-28 to 2017-09-30, total_days 611
unique days in data: 59 (9.66%)


Все увиденное вызывает следующие мысли:
 - Возможно это нормально, что квартиры не продаются каждый день. Но тот факт, что за весь исследуемый временной диапазон (01/08/2016 - 30/09/2017) в данных есть информация по продажам только о 59 днях (9.66%), как-то настораживает. Кажется, что для города с населением больше 5млн. чел. это мало. Поэтому в этом месте возникает гипотеза - нам выдали сильно урезанные данные.
 - Будем надеяться, что CatBoost сможет найти ценность в новых признаках: month и day_name

In [80]:
# Чтобы удовлетворить любопытство, посмотрим ежемесячные тренды по деньгам.
t = train.pivot_table(index='month', aggfunc={'price': ['sum', 'mean']})
t.columns = ['_'.join(col) for col in t.columns]

In [81]:
# Похоже, что какие-то паттерны все же есть.
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=t.index, y=t['price_mean'], name="price_mean"), secondary_y=False)
fig.add_trace(go.Scatter(x=t.index, y=t['price_sum'], name="price_sum"), secondary_y=True)
fig.update_layout(title_text="Динамика изменения цен по месяцам")

### distance

In [82]:
train['distance'].hist()

Уникальных расстояний от объекта до центра достаточно много (210). Попробуем их сгруппировать используя правило Стёрджеса. И для простоты будем чуть-чуть округлять :)
Но начнем с объектов, которые имеют расстояние равное `0`. По странному стечению обстоятельств такие значения (`и в kaggle_test`) только у объектов из района `Melbourne`

In [83]:
def get_distance(data):
    data.loc[(data['suburb'] == 'Melbourne') & (data['distance'] == 0), 'distance'] =
    data[(data['suburb'] == 'Melbourne') & (data['distance'] != 0)]['distance'].mean()

    # Рассчитываем количество групп и шаг между группами
    n = round(1 + np.log2(data['distance'].nunique()))
    h = round((data['distance'].max() - 1) / n)

    bins = []
    for i in range(0, h * (n + 1)):
        if i % 5 == 0:
            bins.append(i)
    data['distance_label'] = pd.cut(data['distance'], bins=bins, labels=bins[1:])
    data.loc[data['distance_label'].isna(), 'distance_label'] = 45
    data['distance_label'] = data['distance_label'].astype('object')
    return data

In [84]:
data = get_distance(train)

### postcode

In [85]:
# Пока нет никаких идей по использованию данного признака.
# Приведем тип данных к целочисленному
train['postcode'] = train['postcode'].astype('int32')

### bedrooms2 / bathroom /buldingarea

In [86]:
# Самое время попробовать разные алгоритмы sklearn по обработке пропусков :)
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

simple = SimpleImputer(strategy='mean')
iterative = IterativeImputer(random_state=42, initial_strategy='mean')

In [87]:
# Simple
train['bedroom2_simple_imputer'] = simple.fit_transform(train[['bedroom2']])
train['bathroom_simple_imputer'] = simple.fit_transform(train[['bathroom']])
train['buildingarea_simple_imputer'] = simple.fit_transform(train[['buildingarea']])
train['car_simple_imputer'] = simple.fit_transform(train[['car']])
train['landsize_simple_imputer'] = simple.fit_transform(train[['landsize']])

In [88]:
# Iterative
imputed = iterative.fit_transform(train[['rooms', 'bedroom2', 'bathroom', 'buildingarea', 'car', 'landsize']])
imputed = pd.DataFrame(imputed, columns=['rooms', 'bedroom2', 'bathroom', 'buildingarea', 'car', 'landsize'])
train['bedroom2_iterative_imputer'] = imputed['bedroom2']
train['bathroom_iterative_imputer'] = imputed['bathroom']
train['buildingarea_iterative_imputer'] = imputed['buildingarea']
train['car_iterative_imputer'] = imputed['car']
train['landsize_iterative_imputer'] = imputed['landsize']

In [88]:
# TODO:
# - Восстановить пропуски руками, через группы
# - Сделать новые метрики: отношение спален к комнатам или к общей площади дома
# - Посмотреть корреляцию. Есть ли там сильные линейные связи?
# - Попробовать через Pipeline Sklearn

---

### Пробуем

In [89]:
pd.options.display.float_format = '{:,.8f}'.format

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error  # as sk_mape
from catboost import CatBoostRegressor

In [91]:
history = {}

In [92]:
def error(y_true, y_pred):
    history = {
        'v1': {'mae': 156624.9525270909, 'mape': 0.1476833053882142, 'leaderboard': 15.22102}
    }

    v1 = 'pred_all_data_Price'
    print(f"now: {mean_absolute_error(y_true, y_pred)} /// v1: {history['v1']['mae']}")
    print(
        f"now: {mean_absolute_percentage_error(y_true, y_pred)} /// v1: {history['v1']['mape']} /// leaderboard: {history['v1']['leaderboard']}")

In [93]:
def get_info(val=True):
    best_iteration = model.get_best_iteration()
    learn = float(model.get_best_score()['learn']['MAPE'])
    if val:
        validation = float(model.get_best_score()['validation']['MAPE'])
    else:
        validation = 'not validation'
    print(f'best_iteration: {best_iteration}, learn: {learn}, val: {validation}')

In [94]:
convert_nan = ['suburb', 'address', 'rooms', 'type', 'method', 'sellerg', 'councilarea', 'regionname']
for i in convert_nan:
    train[i].fillna('no_info', inplace=True)

In [95]:
train_, test_ = train_test_split(train, train_size=0.6, random_state=42)
test_, val_ = train_test_split(test_, train_size=0.5, random_state=42)

In [96]:
train.columns

Index(['suburb', 'address', 'rooms', 'type', 'price', 'method', 'sellerg',
       'date', 'distance', 'postcode', 'bedroom2', 'bathroom', 'car',
       'landsize', 'buildingarea', 'yearbuilt', 'councilarea', 'lattitude',
       'longtitude', 'regionname', 'propertycount', 'id', 'house', 'street',
       'month', 'day_name', 'distance_label', 'bedroom2_simple_imputer',
       'bathroom_simple_imputer', 'buildingarea_simple_imputer',
       'car_simple_imputer', 'landsize_simple_imputer',
       'bedroom2_iterative_imputer', 'bathroom_iterative_imputer',
       'buildingarea_iterative_imputer', 'car_iterative_imputer',
       'landsize_iterative_imputer'],
      dtype='object')

In [138]:
X = [
    'suburb', 'address', 'rooms', 'type', 'method', 'sellerg',
    'date', 'distance', 'postcode',
    # 'bedroom2', 'bathroom', 'car', 'landsize',
    'buildingarea', 'yearbuilt', 'councilarea', 'lattitude',
    'longtitude', 'regionname', 'propertycount', 'id', 'house', 'street',
    'month', 'day_name', 'distance_label',
    # 'bedroom2_simple_imputer', 'bathroom_simple_imputer', 'buildingarea_simple_imputer', #'car_simple_imputer', 'landsize_simple_imputer',
    'bedroom2_iterative_imputer', 'bathroom_iterative_imputer', 'buildingarea_iterative_imputer', 'car_iterative_imputer', 'landsize_iterative_imputer'
]

cat_features = ['suburb', 'address', 'rooms', 'type', 'method', 'sellerg',
                'postcode', 'councilarea', 'regionname', 'house', 'street', 'day_name']

y = ['price']

In [139]:
parameters = {
    'cat_features': cat_features,
    'eval_metric': 'MAPE',
    'verbose': False,
    'random_seed': 42,
    'learning_rate': 0.05}

In [140]:
model = CatBoostRegressor(**parameters)
model.fit(train_[X], train_[y], eval_set=(val_[X], val_[y]))
get_info(val=True)

best_iteration: 994, learn: 0.13242036712814878, val: 0.15553715457730982


In [141]:
# baseline = 0.13811551913729647
test_['pred_price'] = model.predict(test_[X])
mean_absolute_percentage_error(test_['price'], test_['pred_price'])

0.1496313050476703

## Пробуем на всех данных

In [142]:
train_full = pd.concat([train_, val_])

In [143]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'verbose': False,
              'random_seed': 42,
              'iterations': model.best_iteration_ + 1}

In [144]:
model = CatBoostRegressor(**parameters)
model.fit(train_full[X], train_full[y])
get_info(val=False)

best_iteration: None, learn: 0.13056709414202286, val: not validation


In [145]:
# baseline = 0.13228562910628644
test_['pred_price_train_full'] = model.predict(test_[X])
mean_absolute_percentage_error(test_['price'], test_['pred_price_train_full'])

0.14573774881332896

In [147]:
import shap
shap.initjs()

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: numpy.core.multiarray failed to import

## А теперь kaggle_test

In [105]:
kaggle_test = pd.read_csv('../data/kaggle_leopard_challenge_test.csv')
kaggle_test.columns = kaggle_test.columns.str.lower()

In [106]:
kaggle_test = get_address(kaggle_test)

In [107]:
# Преобразуем дату и сгенерируем дополнительные признаки
kaggle_test['date'] = pd.to_datetime(kaggle_test['date'], errors='coerce', format='%d/%m/%Y')
kaggle_test['month'] = kaggle_test['date'].apply(lambda x: x.replace(day=1))
kaggle_test['day_name'] = kaggle_test['date'].dt.day_name()

kaggle_test['date'].isna().mean()

0.0

In [108]:
kaggle_test = get_distance(kaggle_test)

In [109]:
# Simple
kaggle_test['bedroom2_simple_imputer'] = simple.fit_transform(kaggle_test[['bedroom2']])
# Iterative
# Допустим, что пропущенное количество комнат хорошо бы восстанавливать по: rooms, price, но в тестовых данных нет прайса и это может очень сильно испортить всю историю (
imputed = iterative.fit_transform(kaggle_test[['rooms', 'bedroom2']])
imputed = pd.DataFrame(imputed, columns=['rooms', 'bedroom2'])
kaggle_test['bedroom2_iterative_imputer'] = imputed['bedroom2']

In [110]:
# Simple
kaggle_test['bathroom_simple_imputer'] = simple.fit_transform(kaggle_test[['bathroom']])
# Iterative
# Допустим, что пропущенное количество ванных комнат хорошо бы восстанавливать по: rooms, price, но в тестовых данных нет прайса и это может очень сильно испортить всю историю (
imputed = iterative.fit_transform(kaggle_test[['rooms', 'bathroom']])
imputed = pd.DataFrame(imputed, columns=['rooms', 'bathroom'])
kaggle_test['bathroom_iterative_imputer'] = imputed['bathroom']

In [111]:
# Simple
kaggle_test['postcode'] = simple.fit_transform(kaggle_test[['postcode']])
kaggle_test['postcode'] = kaggle_test['postcode'].astype('int32')

In [112]:
convert_nan = ['suburb', 'address', 'rooms', 'type', 'method', 'sellerg', 'councilarea', 'regionname']
for i in convert_nan:
    kaggle_test[i].fillna('no_info', inplace=True)

In [113]:
kaggle_test['Price'] = model.predict(kaggle_test[X])

KeyError: "['buildingarea_simple_imputer', 'buildingarea_iterative_imputer', 'car_iterative_imputer', 'landsize_iterative_imputer'] not in index"

In [ ]:
result = kaggle_test[['id', 'Price']].copy()
result['Price'] = result['Price'].astype('int64')
result

In [ ]:
result.to_csv('leopard_result_v2.csv', index=False)

In [ ]:
# now: 157898.91491277708 /// v1: 156624.9525270909
# now: 0.1477508965159632 /// v1: 0.1476833053882142 /// leaderboard: 15.22102